In [7]:
# Load modules, mainly huggingface basic model handlers.
# Make sure you install huggingface and other packages properly.
from collections import Counter
import json
import copy
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef

import logging
logger = logging.getLogger(__name__)

import os
os.environ["TRANSFORMERS_CACHE"] = "../huggingface_cache/" # Not overload common dir 
                                                           # if run in shared resources.

import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import torch
import argparse
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from datasets import Dataset
from datasets import DatasetDict

import transformers
from transformers import (
    AutoModelForMaskedLM,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback
)
from transformers.trainer_utils import is_main_process, EvaluationStrategy
from functools import partial

In [8]:
# this file contains data preprocessor for the following GLUE datasets.
# CoLA(!), SST(!), MRPC(!), STS-B, QQP, MNLI-m, MNLI-mm, QNLI(!), RTE, WNLI

# the following dataset are not preprocessed yet.
# STS-B, QQP, MNLI-m, MNLI-mm, RTE, WNLI

In [60]:
glue_task = "sst2"

In [61]:
dataset = load_dataset(
    "glue", glue_task,
    cache_dir="../../huggingface_cache/"
)

Dataset glue downloaded and prepared to ../../huggingface_cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [62]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 1821
    })
})

In [63]:
dataset["train"][0]

{'idx': 0,
 'label': 0,
 'sentence': 'hide new secretions from the parental units '}

In [64]:
dataset.save_to_disk(f"../../data-files/{glue_task}")